## Experiments with TEP data and inductive conformal regression

In [1]:
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

from models import ICPRegressionEnsemble, ICPRegressionMCDropout, PowerMartingale, ICPNearestNeighborDist
from utils import TEP_DataLoader
from utils import TEPDATASETTEST, TEPDATASETTRAIN

In [2]:
dataloader = TEP_DataLoader()
dataloader.load_training_data()

In [3]:
X, y = dataloader.get_continuous_dataset(length=100000, target_value="xmeas_35")
X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.50, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=42)

In [4]:
ensemble = ICPRegressionEnsemble(n_members=20, input_shape=(X_train.shape[-1], 1))
ensemble.fit(X_train, y_train)
ensemble.save()
ensemble.load()
ensemble.calibrate(X_val)

Epoch 1/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2090
Epoch 2/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0044
Epoch 3/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0039
Epoch 4/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0036
Epoch 5/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0035
Epoch 6/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0036
Epoch 7/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0036
Epoch 8/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0037
Epoch 9/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0037
Epoch 10/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0037
Epoch 1/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2187
Epoch 2/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0052
Epo

In [5]:
nn = ICPNearestNeighborDist()
nn.fit(X_train)
nn.calibrate(X_val)

In [6]:
mc_dropout = ICPRegressionMCDropout(n_forward_passes=20, input_shape=(X_train.shape[-1], 1))
mc_dropout.fit(X_train, y_train)
mc_dropout.save()
mc_dropout.load()
mc_dropout.calibrate(X_val)

Epoch 1/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2076
Epoch 2/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0044
Epoch 3/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0039
Epoch 4/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0036
Epoch 5/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0035
Epoch 6/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0036
Epoch 7/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0036
Epoch 8/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0037
Epoch 9/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0037
Epoch 10/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.0037


In [7]:
nn_pmm = PowerMartingale(model=nn, alarm_threshold=20)
ensemble_pmm = PowerMartingale(model=ensemble, alarm_threshold=20)
mc_dropout_pmm = PowerMartingale(model=mc_dropout, alarm_threshold=20)

In [10]:
for fault_id in [1, 2, 5, 6, 8, 10, 12, 13]:
    runs_X, runs_y = dataloader.get_runs_dataset(length=20, fault=fault_id, target_value="xmeas_35")

    first_alarm_steps_ensemble = []
    false_negatives_ensemble = 0

    first_alarm_steps_knn = []
    false_negatives_knn = 0

    first_alarm_steps_mc_dropout = []
    false_negatives_mc_dropout = 0

    for i in range(runs_X.shape[0]):
        _, _, first_alarm_step = ensemble_pmm.score(runs_X[i])
        if first_alarm_step is not None:
            first_alarm_steps_ensemble.append(first_alarm_step)
        else:
            false_negatives_ensemble += 1
        _, _, first_alarm_step = nn_pmm.score(runs_X[i])
        if first_alarm_step is not None:
            first_alarm_steps_knn.append(first_alarm_step)
        else:
            false_negatives_knn += 1

        _, _, first_alarm_step = mc_dropout_pmm.score(runs_X[i])
        if first_alarm_step is not None:
            first_alarm_steps_mc_dropout.append(first_alarm_step)
        else:
            false_negatives_mc_dropout += 1

    print("Fault Type: " + str(fault_id))
    print("Average delay Ensemble: " + str(np.mean(first_alarm_steps_ensemble)))
    print("Average delay KNN: "+ str(np.mean(first_alarm_steps_knn)))
    print("Average delay MCD: "+ str(np.mean(first_alarm_steps_mc_dropout)))

    print("False negatives Ensemble: " + str(false_negatives_ensemble))
    print("False negatives KNN: "+ str(false_negatives_knn))
    print("False negatives MCD: "+ str(false_negatives_mc_dropout))
    

Fault Type: 1
Average delay Ensemble: 32.75
Average delay KNN: 29.65
Average delay MCD: 33.1
False negatives Ensemble: 0
False negatives KNN: 0
False negatives MCD: 0
Fault Type: 2
Average delay Ensemble: 59.2
Average delay KNN: 51.7
Average delay MCD: 74.3
False negatives Ensemble: 0
False negatives KNN: 0
False negatives MCD: 0
Fault Type: 5
Average delay Ensemble: 35.85
Average delay KNN: 36.4
Average delay MCD: 46.0
False negatives Ensemble: 0
False negatives KNN: 0
False negatives MCD: 0
Fault Type: 6
Average delay Ensemble: 45.25
Average delay KNN: 33.05
Average delay MCD: 46.5
False negatives Ensemble: 0
False negatives KNN: 0
False negatives MCD: 0
Fault Type: 8
Average delay Ensemble: 61.2
Average delay KNN: 49.5
Average delay MCD: 63.8
False negatives Ensemble: 0
False negatives KNN: 0
False negatives MCD: 0
Fault Type: 10
Average delay Ensemble: 215.41666666666666
Average delay KNN: 227.11111111111111
Average delay MCD: 215.33333333333334
False negatives Ensemble: 8
False ne